In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.utils import shuffle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

In [30]:
df = pd.read_csv('/content/drive/My Drive/result1.csv')
df[144:154] 

,_id,topic,heading,summary,date_published,author,content,tags
144,5efff12b86936ae10d2be3b9,NaN,"[""\n Chill, Focus, Energy, or...","[""\n Although YouTube has not...",NaN,NaN,"[""YouTube Music is saving you time to search f...",NaN
145,5efff12d86936ae10d2be3ba,"[""delhi""]","[""Centre first opposed, then copied Delhi’s is...","[""AAP’s national spokesperson and MLA Raghav C...","[""Updated: Jul 04, 2020 02:35 IST""]","[""HT Correspondent""]","[""The Aam Aadmi Party (AAP) on Friday alleged ...","[""BJP"",""Aam Aadmi Party"",""Delhi government"",""A..."
146,5efff12d86936ae10d2be3bb,NaN,"[""\n Tech Wrap: Jio Platforms...","[""\n More tech news include L...",NaN,NaN,"[""The day started with Jio Platforms bagging y...",NaN
147,5efff13086936ae10d2be3bc,"[""delhi""]","[""Delhi cop dies after being hit by speeding c...","[""Assistant sub inspector Lal Man Singh Sisodi...","[""Updated: Jul 04, 2020 00:57 IST""]","[""Anvit Srivastava""]","[""A 51-year-old assistant sub-inspector(ASI) o...","[""United States embassy"",""Delhi"",""Delhi Police""]"
148,5efff13286936ae10d2be3bd,"[""education""]","[""MP Board MPBSE 10th Result 2020 to be declar...","[""MP Board 10th Result 2020: Madhya Pradesh Bo...","[""Updated: Jul 04, 2020 07:16 IST""]","[""hindustantimes.com| Edited by Nandini""]","[""MP Board 10th Result 2020: Madhya Pradesh Bo...",NaN
149,5efff13486936ae10d2be3be,NaN,"[""MP Board 10th Result 2020 Live Updates: Madh...","[""MP Board 10th Result 2020: Passing marks""]",NaN,NaN,NaN,NaN
150,5efff13786936ae10d2be3bf,"[""education""]","[""MP Board 10th result 2020 soon: Check MPBSE ...","[""MP 10th result 2020: Students who have appea...","[""Updated: Jul 04, 2020 07:26 IST""]","[""hindustantimes.com | Edited by Akhilesh Naga...","[""MP 10th result 2020:"","" Madhya Pradesh Board...","[""Madhya Pradesh"",""Board exam"",""Result""]"
151,5efff13a86936ae10d2be3c0,"[""business""]","[""76 taxpayers per minute: Rs 62K crore I-T re...","[""I-T refunds amounting to Rs 23,453.57 crore ...","[""Updated: Jul 03, 2020 23:27 IST""]","[""HT Correspondent | Edited by Arpan Rai""]","[""The Income Tax (I-T) department has issued t...","[""Tax refund"",""CBDT"",""Finance Ministry""]"
152,5efff13c86936ae10d2be3c1,"[""india""]","[""GST regime may undergo 2 major reforms this...","[""GST was launched in 2017 as India’s biggest-...","[""Updated: Jul 04, 2020 07:27 IST""]","[""Rajeev Jayaswal""]","[""The Goods and Services Tax (GST) regime is e...","[""GST"",""finance ministry""]"
153,5efff13f86936ae10d2be3c2,"[""business""]","[""Services sector’s PMI shows slight uptick in...","[""According to data released by IHS Markit, se...","[""Updated: Jul 04, 2020 08:06 IST""]","[""Asit Ranjan Mishra""]","[""Services sector activity, which makes up mor...","[""Services sector"",""India"",""coronavirus"",""COVI..."


In [ ]:
len(df.content.unique()),  len(df.content)

(154, 162)

In [ ]:
df.content[0]

'["Prime Minister (PM) Narendra Modi recently announced that India must become ","(self-reliant). One aspect of this could be that India will remove barriers within its internal markets to truly become a single market. It will remove the hurdles to efficiency improvements and become more competitive. The Goods and Services Tax (GST) was a step in this direction. Recent decisions to remove hurdles in inter-state agricultural trade are also similar. For agricultural and industrial products, as well as capital, India is increasingly becoming a single market. The creation of a barrier-free domestic market is also an intent reflected in Article 301 of the Constitution.","However, there is one market where frictions are being added rather than reduced. This is the labour market. For different reasons, leaders from out-migration and in-migration states have made statements suggesting that there may be more impediments to the inter-state migration of workers. Some states have announced prefere

In [ ]:
# time range for which these articles are published
#min(df.date_published).split("T")[0], max(df.date_published).split("T")[0]

In [ ]:
# the number of articles in each section
Counter(df.topic)

Counter({'["analysis"]': 11,
         '["art-and-culture"]': 1,
         '["bollywood"]': 7,
         '["business"]': 9,
         '["cities"]': 1,
         '["columns"]': 4,
         '["cricket"]': 4,
         '["delhi"]': 8,
         '["editorials"]': 9,
         '["education"]': 13,
         '["fashion-and-trends"]': 4,
         '["fitness"]': 1,
         '["football"]': 3,
         '["health"]': 1,
         '["hollywood"]': 1,
         '["india"]': 17,
         '["it-s-viral"]': 12,
         '["more-lifestyle"]': 4,
         '["mumbai"]': 10,
         '["music"]': 1,
         '["opinion"]': 2,
         '["other-sports"]': 4,
         '["regional-movies"]': 1,
         '["tennis"]': 1,
         '["travel"]': 1,
         '["tv"]': 12,
         '["world"]': 11,
         nan: 9})

In [ ]:
df.isnull().sum()

_id                   0
topic                 9
heading               0
summary               0
date_published        9
author               20
content               7
tags                 27
processed_heading     0
dtype: int64

In [5]:
df.dropna(subset=['content'], inplace = True)
df.dropna(subset = ['topic'], inplace = True)

In [6]:
df.shape

(148, 8)

In [31]:
df[-10:]

,_id,topic,heading,summary,date_published,author,content,tags
152,5efff13c86936ae10d2be3c1,"[""india""]","[""GST regime may undergo 2 major reforms this...","[""GST was launched in 2017 as India’s biggest-...","[""Updated: Jul 04, 2020 07:27 IST""]","[""Rajeev Jayaswal""]","[""The Goods and Services Tax (GST) regime is e...","[""GST"",""finance ministry""]"
153,5efff13f86936ae10d2be3c2,"[""business""]","[""Services sector’s PMI shows slight uptick in...","[""According to data released by IHS Markit, se...","[""Updated: Jul 04, 2020 08:06 IST""]","[""Asit Ranjan Mishra""]","[""Services sector activity, which makes up mor...","[""Services sector"",""India"",""coronavirus"",""COVI..."
154,5efff14286936ae10d2be3c3,"[""hollywood""]","[""When Amrish Puri refused to audition for Ste...","[""Did you know that Steven Spielberg once desc...","[""Updated: Jul 03, 2020 17:13 IST""]","[""HT Entertainment Desk""]","[""Several Indian actors have made their mark i...","[""Steven Spielberg"",""Amrish Puri"",""Temple of D..."
155,5efff14586936ae10d2be3c4,"[""bollywood""]","[""‘Good roles, good films reserved for people ...","[""Celina Jaitly says most of the meaty parts a...","[""Updated: Jul 04, 2020 07:08 IST""]","[""Ruchi Kaushal""]","[""Celina Jaitly"","" has said she had earlier di...","[""Celina Jaitly"",""Rituporno Ghosh"",""Seasons Gr..."
156,5efff14786936ae10d2be3c5,"[""tv""]","[""Smriti Irani gets nostalgic as Kyunki Saas B...","[""Actor and minister Smriti Irani went back in...","[""Updated: Jul 04, 2020 08:05 IST""]","[""HT Entertainment Desk""]","[""Actor and minister "",""Smriti Irani"","" went b...","[""Smriti Irani"",""Ekta Kapoor"",""Kyunki Saas Bhi..."
157,5efff14986936ae10d2be3c6,"[""other-sports""]","[""Can F1’s young guns dethrone King Lewis?""]","[""OPENER: Leclerc-Verstappen duels could signa...","[""Updated: Jul 04, 2020 06:53 IST""]","[""Rajesh Pansare""]","[""Last year’s Austrian Grand Prix may have giv...","[""Lewis Hamilton"",""Max Verstappen"",""Charles Le..."
158,5efff14c86936ae10d2be3c7,"[""other-sports""]","[""Hamilton makes a statement as F1 gets back o...","[""Hamilton, wearing a ‘Black Lives Matter’ hel...","[""Updated: Jul 04, 2020 07:02 IST""]","[""Reuters""]","[""Formula One got back on track in changed cir...","[""Lewis Hamilton"",""Mercedes"",""Valtteri Bottas""..."
159,5efff14f86936ae10d2be3c8,"[""football""]","[""Morata double leads Atletico to 3-0 win over...","[""Morata earned a penalty when he was pulled d...","[""Updated: Jul 04, 2020 07:12 IST""]","[""Associated Press""]","[""Alvaro Morata scored twice to lead Atletico ...","[""Alvaro Morata"",""Mallorca"",""Diego Simeone"",""R..."
160,5efff15286936ae10d2be3c9,"[""it-s-viral""]","[""Twitter can’t decide if this puzzle related ...","[""Shared on Twitter, the puzzle related video ...","[""Updated: Jul 03, 2020 18:15 IST""]","[""Trisha Sengupta""]","[""Have you ever solved a picture puzzle? For m...","[""Video"",""Twitter""]"
161,5efff15386936ae10d2be3ca,"[""it-s-viral""]","[""Mum records sweet interaction between daught...","[""Eight-year-old, Tallulah drew a thank you no...","[""Updated: Jul 03, 2020 18:17 IST""]","[""Sanya Budhiraja""]","[""The beautiful bond of friendship can genuine...","[""friendship"",""viral video""]"


In [7]:
import pandas as pd
import numpy as np
import re

import gensim
from gensim import corpora,models
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

import nltk
#nltk.download('wordnet')
#nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
# data preprocessing
stemmer = SnowballStemmer(language='english',ignore_stopwords=True)
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [9]:
# preview after preprocessing
doc_sample = df['content'][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['["Prime', 'Minister', '(PM)', 'Narendra', 'Modi', 'recently', 'announced', 'that', 'India', 'must', 'become', '","(self-reliant).', 'One', 'aspect', 'of', 'this', 'could', 'be', 'that', 'India', 'will', 'remove', 'barriers', 'within', 'its', 'internal', 'markets', 'to', 'truly', 'become', 'a', 'single', 'market.', 'It', 'will', 'remove', 'the', 'hurdles', 'to', 'efficiency', 'improvements', 'and', 'become', 'more', 'competitive.', 'The', 'Goods', 'and', 'Services', 'Tax', '(GST)', 'was', 'a', 'step', 'in', 'this', 'direction.', 'Recent', 'decisions', 'to', 'remove', 'hurdles', 'in', 'inter-state', 'agricultural', 'trade', 'are', 'also', 'similar.', 'For', 'agricultural', 'and', 'industrial', 'products,', 'as', 'well', 'as', 'capital,', 'India', 'is', 'increasingly', 'becoming', 'a', 'single', 'market.', 'The', 'creation', 'of', 'a', 'barrier-free', 'domestic', 'market', 'is', 'also', 'an', 'intent', 'reflected', 'in', 'Article', '301', 'of', 'the', 'Constitution."

In [10]:
# Preprocess the headline text, saving the results as ‘processed_docs’
df['processed_heading'] = df['heading'].map(preprocess)
df['processed_content'] = df['content'].map(preprocess)
df['processed_topic'] = df['topic'].map(preprocess)
df['processed_heading'] = df['processed_heading'].apply(lambda x: ' '.join(x))
df['processed_content'] = df['processed_content'].apply(lambda x: ' '.join(x))
df['processed_topic'] = df['processed_topic'].apply(lambda x: ' '.join(x))
df.head() 

,_id,topic,heading,summary,date_published,author,content,tags,processed_heading,processed_content,processed_topic
0,5ef0d2b4d83b3587cb500ec9,"[""analysis""]","[""Don’t discriminate against non-resident migr...","[""Create a legal regime that allows them to ac...","[""Updated: Jun 22, 2020 19:30 IST""]","[""KP Krishnan and Anirudh Burman""]","[""Prime Minister (PM) Narendra Modi recently a...","[""India"",""Narendra Modi"",""BIHAR"",""Haryana"",""Mi...",discrimin resid migrant opinion,prime minist narendra modi recent announc indi...,analysi
1,5ef0d2b8d83b3587cb500ecb,"[""opinion""]","[""India has closed military gap with China alo...","[""Over the past dozen years India has not only...","[""Updated: Jun 22, 2020 16:01 IST""]","[""Pramit Pal Chaudhuri""]","[""India has the "","", says a recent Harvard Uni...","[""Indian Army"",""Galwan Valley"",""India China bo...",india close militari china border,india say recent harvard univers assess chines...,opinion
2,5ef0d2bad83b3587cb500ecc,"[""opinion""]","[""Ladakh faceoff: Looking back for a way forwa...","[""DRAW THE LINE Policing the borders and preve...","[""Updated: Jun 22, 2020 18:06 IST""]","[""Lt Gen GS Sihota (retd)""]","[""On December 11, 1917, General Edmund Allenby...",NaN,ladakh faceoff look forward,decemb general edmund allenbi enter holi citi ...,opinion
3,5ef0d2bdd83b3587cb500ecd,"[""editorials""]","[""Redesign the model village scheme | HT Edito...","[""For MPs to take it up, expand its ambit to a...","[""Updated: Jun 22, 2020 18:56 IST""]",NaN,"[""A study commissioned by the Union ministry o...",NaN,redesign model villag scheme editori,studi commiss union ministri rural develop saa...,editori
4,5ef0d2bed83b3587cb500ece,"[""columns""]","[""How China has overplayed its hand, writes Ba...","[""Beijing’s belligerence reflects a familiar h...","[""Updated: Jun 22, 2020 19:31 IST""]","[""Baijayant “Jay” Panda""]","[""Almost 2,500 years ago, Greek historian, Thu...","[""CHINA"",""India""]",china overplay hand write baijay panda,year greek historian thucydid write devast pel...,column


In [11]:
#Split the data into a "train" and "test". As well as select a metric in which to measure the similarity between articles. Think of the "train" set as the corpus.
# Think of the "test" set as the NYT articles that users are currently reading.

# move articles to an array
articles = df.processed_content.values

# move article section names to an array
sections = df.processed_topic.values

# move article web_urls to an array
headings = df.processed_heading.values

# shuffle these three arrays 
articles, sections, headings = shuffle(articles, sections, headings, random_state=4)

In [12]:
# split the shuffled articles into two arrays
n = 10

# one will have all but the last 10 articles -- think of this as your training set/corpus 
X_train = articles[:-n]
X_train_headings = headings[:-n]
X_train_sections = sections[:-n]

# the other will have those last 10 articles -- think of this as your test set/corpus 
X_test = articles[-n:]
X_test_headings = headings[-n:]
X_test_sections = sections[-n:]

In [13]:
# instantiate your vectorizor 
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

In [14]:
# fit the vectorizer 
tfidf_vectorizer.fit(X_train)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [15]:
# transform both article splits 
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [16]:
#Builidng a Content Based Recommender
def get_top_n_rec_articles(X_train_tfidf, X_train, test_article, X_train_sections, X_train_headings, n = 5):
    '''This function calculates similarity scores bewteen a document and a corpus
    
       INPUT: vectorized document corpus, 2D array
              text document corpus, 1D array
              user article, 1D array
              article section names, 1D array
              article URLs, 1D array
              number of articles to recommend, int
              
       OUTPUT: top n recommendations, 1D array
               top n corresponding section names, 1D array
               top n corresponding URLs, 1D array
               similarity scores bewteen user article and entire corpus, 1D array
              '''
    # calculate similarity between the corpus (i.e. the "test" data) and the user's article
    similarity_scores = X_train_tfidf.dot(test_article.toarray().T)

    # get sorted similairty score indicies 
    sorted_indicies = np.argsort(similarity_scores, axis = 0)[::-1]

    # get sorted similarity socres
    sorted_sim_scores = similarity_scores[sorted_indicies]

    # get top n most similar documents
    top_n_recs = X_train[sorted_indicies[:n]]

    # get top n corresponding document section names
    rec_sections = X_train_sections[sorted_indicies[:n]]

    # get top n corresponding urls
    rec_headings = X_train_headings[sorted_indicies[:n]]
    
    # return recommendations and corresponding article meta-data
    return top_n_recs, rec_sections, rec_headings, sorted_sim_scores

In [50]:
# pick an article from the "test" set
# treat this as the article that the user is currently reading
k = 6
test_article = X_test_tfidf[k]

In [49]:
# user's article's section name
X_test_sections[6]

'educ'

In [51]:
# user's article
X_test[k]

'awar curv covid pandem flatten soon state govern ask higher secondari school degre colleg start onlin class juli ask prepar onlin content student director higher secondari educ ask princip higher secondari school state explor onlin method instead chalk talk method break rule advisori covid guidelin author undergradu colleg ask whatsapp mail group student exceed student teach order onlin coach come state govern extend closur school colleg univers till august view rise covid case state june total number case death jump time rise case council higher secondari educ wednesday cancel pend paper higher secondari examin govern say teacher provid soft copi note text book ppts recommend lectur whatsapp mail studi home teacher conduct onlin class student hour day week googl meet zoom skype digit platform onlin class conduct colleg premis teacher maintain attend regist student attend onlin class prepar time tabl class order higher educ depart say offici say govern univers colleg carri offici work

In [52]:
# return the top n most similar articles as recommendations 
top_n_recs, rec_sections, rec_headings, sorted_sim_scores = \
get_top_n_rec_articles(X_train_tfidf, X_train,  test_article,X_train_sections, X_train_headings, n = 5 )

In [53]:
#validate the results
#Compare the user's article and corresponding section name with the recommended articles and corresponding section names

# similarity scores
sorted_sim_scores[:5]

array([[[0.37007318]],

       [[0.32363991]],

       [[0.30611753]],

       [[0.30222279]],

       [[0.28436095]]])

In [36]:
# user's article
X_test[k]

'extend provis free ration million citizen newspap welcom prime minist narendra modi refer festiv season mention chhath festiv bihar fact elect bihar year scheme get extend till novemb prompt suggest polit motiv drive decis west bengal chief minist mamata banerje say govern provid free ration till june coincid elect west bengal year democraci poor marginalis leverag power vote forc govern elit come welfar scheme address distress poor scheme mahatma gandhi nation rural employ guarante scheme case congress provis cylind hous toilet incom transfer farmer case bharatiya janata parti help parti elect deliv poor reward welfar laudabl essenti indian polit discours move challeng current time employ creation pandem exacerb crisi employ particular urban area time polit parti compet basi provid better welfar scheme creat job leader invest sound econom manag realis lead advers elector outcom'

In [37]:
# user's article's section name
X_test_sections[k]

'editori'

In [54]:
# corresponding section names for top n recs 
rec_sections

array([['educ'],
       ['india'],
       ['analysi'],
       ['educ'],
       ['educ']], dtype=object)

In [55]:
# top n article recs
top_n_recs

array([['govern postpon juli decis reopen school state instead encourag teacher hold class onlin student order engag chief minist pramod sawant announc sawant insist onlin class compulsori syllabus mandatorili cover face face class say govern later decid curtail holiday ensur syllabus complet school reopen state train school teacher give class onlin expect come week await reopen school connect parent children smartphon say compulsori onlin educ student miss connect smartphon parent deliber smartphon sawant say say onlin class commenc teacher student engag meet offici educ depart chief minist educ minist repres headmast associ hold monday govern decid mandatori teacher attend school govern servant begin attend offic teacher compulsorili attend school govern servant attend offic teacher sawant say govern decid automat promot student class initi schedul answer supplementari examin need promot class central govern take decis allow reopen school countri case mount guidelin issu unlock clear

In [56]:
# corresonding URLs for top n recs 
rec_headings

array([['onlin class begin compulsori say pramod sawant'],
       ['delhi announc learn human feel bridg digit divid covid crisi'],
       ['covid tech ensur equiti educ'],
       ['plea seek waiver maximum relief school period covid lockdown'],
       ['assam result date time ahsec announc class result june time']],
      dtype=object)